In [15]:
#import libraries
import pandas as pd
import numpy as np
from sklearn.preprocessing import scale
from sklearn.model_selection import train_test_split

In [16]:
#import features for modeling
features = pd.read_csv('Features.csv')
features.head()

,Provider,Phys_Count,Patient_Count,Claim_Count,Both_Service,Inpatient,Outpatient,DuplicateClaims,Duplicate_Claims_Percent,AvgClaimDuration,...,ChronicCond_KidneyDisease,ChronicCond_Cancer,ChronicCond_ObstrPulmonary,ChronicCond_Depression,ChronicCond_Diabetes,ChronicCond_IschemicHeart,ChronicCond_Osteoporasis,ChronicCond_rheumatoidarthritis,ChronicCond_stroke,PotentialFraud
0,PRV51017,114,420,763,0,0,1,496,0.65,1.460026,...,1203,1427,1284,1218,1001,944,1253,1327,1453,No
1,PRV51081,4,131,364,1,0,0,328,0.90,1.634615,...,531,696,627,568,472,427,627,629,678,No
2,PRV51123,21,28,43,1,0,0,24,0.56,3.069767,...,69,85,69,66,52,55,82,71,85,No
3,PRV51139,34,42,60,1,0,0,34,0.57,2.566667,...,95,111,104,85,73,72,103,108,115,No
4,PRV51140,6,394,697,1,0,0,420,0.60,2.226686,...,1112,1282,1156,1095,888,873,1178,1158,1314,No


In [10]:
#scale features
X = features.drop(['Provider', 'PotentialFraud'], axis=1)

scaled_X = scale(X)

y = features['PotentialFraud']

In [13]:
#train/test split 
np.random.seed(0)
X_train, X_test, y_train, y_test = train_test_split(scaled_X, y, test_size=0.2, random_state=42)

In [17]:
#upsampling for inbalanced potential fraud column

features['PotentialFraud'].value_counts()

No     1052
Yes     290
Name: PotentialFraud, dtype: int64